<br>
<font size='6'><b>Advanced Arduino</b></font><br><br>

<table style="border-style: hidden; border-collapse: collapse;" width = "80%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 60% style="border-style: hidden; border-collapse: collapse;">

        </td>
        <td width = 20%>
        by Seungchul Lee<br>iSystems Design Lab<br>http://isystems.unist.ac.kr/<br>UNIST
        </td>
    </tr>
</table>

Table of Contents
<div id="toc"></div>

# 1. Visual Output (revisited)



## 1.1. Dual 7 Segment

We have learned about 7 segments that can only represent 1 digit. However, there are many applications where we need to represent more than one digit. Let's make an embedded system that can represent two digits using a dual 7 segment (two 7 segments).

<img src = "./image_files/2_7seg.jpg" width = 400>

The first thing you’re probably wondering is: “__how can we control a total of 16 different segments (LED) if we only have ten pins on the Arduino?__”. 

To get some hints, see the below exprement.

- space vs. time

In [2]:
%%html
<iframe src="https://www.youtube.com/embed/1lA4UUCGUu0" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

In multiplexed LED applications the LED segments of all the digits are tied together and the common pins of each digit are turned ON separately by the microcontroller. 

When each digit is displayed only for several milliseconds, the eye cannot tell that the digits are not ON all the time. This way we can multiplex any number of 7-segment displays together. 

For example, to display the number 53, we have to send 5 to segments $a$ to $g$ and enable its common pin. After a few milliseconds, number 3 is sent to segments $a$ to $g$ and the common point of the second digit is enabled. When this process is __repeated continuously and fast enough__, it appears to the user that both displays are ON continuously.

 1) Send the segment bit pattern to segments $a$ to $g$<br>
 2) Enable digit 1<br>
 3) Wait for a few milliseconds<br>
 4) Disable digit 1<br>
 5) Send the segment bit pattern to segments $a$ to $g$<br>
 6) Enable digit 2<br>
 7) Wait for a few milliseconds<br>
 8) Disable digit 2<br>
 9) Repeat

<br>
<font size='4'><b>Lab 1: Two Digit Counter</b></font>

<table style="border-style: hidden; border-collapse: collapse;" width = "90%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 25% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/2_7seg.jpg" width = 250>
        </td>
        <td width = 30% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/two_7segs.jpg" width = 250>
        </td>
        <td width = 35%>
<img src = "./image_files/Dual7-segment.jpg" width = 400>
        </td>
    </tr>
</table>

<img src = "./image_files/two_7seg_circuit.png" width = 350>

Add code for displaying 24

<br>
<font size='4'><b>Lab 2: Two Digit Counter from 0 to 99</b></font>

<img src = "./image_files/two_7seg_circuit.png" width = 350>

>need to add code

<font size='4'><b>Lab 3: Use LEDDisplay.h library</b></font>



```c
#include <LEDDisplay.h>

LEDDisplay *led;

void setup() {
  int digitFlagPins[] = {10, 11};
  int segmentPins[] = {2, 3, 4, 5 , 6 , 7 , 8, 9};
  int decimalPointPin = 9;
  led = new LEDDisplay(2, digitFlagPins, segmentPins, decimalPointPin);
}

void loop() {
  led -> displayNumber(2, 1);
  led -> displayNumber(4, 0);
}

```

To see this in a slightly more dynamic example, we can adapt the example to make the LED display act as a counter (obviously only up to the number 99). 

```c
#include <LEDDisplay.h>

LEDDisplay *led;
int millisecondsPerCount;
int counter;
unsigned long lastUpdate;

void setup()  {
  millisecondsPerCount = 1000;
  int digitFlagPins[] = {10, 11};
  int segmentPins[] = {2, 3, 4, 5 , 6 , 7 , 8, 9};
  int decimalPointPin = 9;
  led = new LEDDisplay(2, digitFlagPins, segmentPins, decimalPointPin);
}

void loop()  {
  unsigned long now = millis();
  if (now - lastUpdate > millisecondsPerCount) {
    lastUpdate = now;
    counter++;
    if (counter == 100) {
      counter = 0;
    }
  }

  int number = counter;
  for (int i = 0; i < 2; i++) {
    led -> displayNumber(number % 10, i);
    delay(2);
    number = number / 10;
  }
}
```

## 1.2. POV Display

- Persistence-Of-Vision (POV) display

- It makes use of the fact that our eyes are acting like a low pass filter.


In [3]:
%%html
<iframe src="https://www.youtube.com/embed/Md78sOI1-r8" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

 When a person sees an object, its image remains in the retina of the eye for a time interval of 1/16th of a second. This phenomenon is known as persistence of vision of the eye. This phenomenon is used in the POV Display to form images. We turn the LEDs on and off in such a way that the different images overlap each other forming letters. For example:
 
<img src = "./image_files/POV-Display-min.jpg" width = 500>

In [4]:
%%html
<iframe src="https://www.youtube.com/embed/57carjNxI9A" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

# 2. Serial Communication

## 2.1. What's happening during serial communication?

In [5]:
%%html
<iframe src="https://player.vimeo.com/video/97519477" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

## 2.2. Multiple values in seiral communication

In [6]:
%%html
<iframe src="https://player.vimeo.com/video/97527889" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

<font size='4'><b>`Serial.write()`</b></font>

- Writes binary data to the serial port. This data is sent as a byte or series of bytes; 
- to send the characters representing the digits of a number use the `print()` function instead.

```c
void setup() {
  Serial.begin(9600);
}

void loop() {
  Serial.write(45); // send a byte with the value 45
  int bytesSent = Serial.write("hello"); //send the string “hello” and return the length of the string.
}
```

- Serial.write(val)
    - val: a value to send as a single byte


- Serial.write(str)
    - str: a string to send as a series of bytes
    

- Serial.write(buf, len)
    - buf: an array to send as a series of bytes
    - len: the length of the buffer 

<br>
<font size='4'><b>`serialEvent()`</b></font>

- Called when data is available. Use Serial.read() to capture this data.

```c
void serialEvent() {
  //statements
}
```


```c
/*
  Serial Event example

 When new serial data arrives, this sketch adds it to a String.
 When a newline is received, the loop prints the string and
 clears it.

 A good test for this is to try it with a GPS receiver
 that sends out NMEA 0183 sentences.

 Created 9 May 2011
 by Tom Igoe

 This example code is in the public domain.

 http://www.arduino.cc/en/Tutorial/SerialEvent

 */

String inputString = "";         // a string to hold incoming data
boolean stringComplete = false;  // whether the string is complete

void setup() {
  // initialize serial:
  Serial.begin(9600);
  // reserve 200 bytes for the inputString:
  inputString.reserve(200);
}

void loop() {
  // print the string when a newline arrives:
  if (stringComplete) {
    Serial.println(inputString);
    // clear the string:
    inputString = "";
    stringComplete = false;
  }
}

/*
  SerialEvent occurs whenever a new data comes in the
 hardware serial RX.  This routine is run between each
 time loop() runs, so using delay inside loop can delay
 response.  Multiple bytes of data may be available.
 */
void serialEvent() {
  while (Serial.available()) {
    // get the new byte:
    char inChar = (char)Serial.read();
    // add it to the inputString:
    inputString += inChar;
    // if the incoming character is a newline, set a flag
    // so the main loop can do something about it:
    if (inChar == '\n') {
      stringComplete = true;
    }
  }
}
```

<font size='4'><b>print string</b></font>

```c
void setup() {
  Serial.begin(9600);
}

void loop() {
  String msg = "";
  if (Serial.available() > 0) {
    while (Serial.available() > 0) {
      msg += char(Serial.read());
      delay(5);
    }
    Serial.println(msg);
  }
}
```



# 3. Interrupt 

You want to perform some action when a digital pin changes value and you don’t want to constantly check the pin state.

## 3.1. External Interrupt

__Interrupts vs. Polling__

- Polling:
    - Continuously poll IOs for change of value
    - Polling is like picking up your phone every few seconds to see if you have a call...
    - Cons:
        - Most polls are unneeded – value did not change
        - High CPU usage
        - Reaction time depends on #IOs

- Interrupt
    - Normal execution is interrupted when event occurs
    - Interrupts are like waiting for the phone to ring
    - Time spent in interrupt handlers should be kept as short as possible
    - Pro:
        - Processor resources are only used when necessary
    - Cons:
        - Program execution is interrupted in a non-deterministic manner


<img src = "./image_files/interrupt_flow_02.png" style="border:1px solid black", width =500>

__Interrupt Service Routine (ISR)__

- Event handler for interrupt
- Special, user-defined function for handling the interrupt

Watch the below introductory lecture on interrupt.

In [7]:
%%html
<iframe src="https://www.youtube.com/embed/M73dNDkBeT8" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

Add circuit scheme and arduino code

<font size='4'><b>Lab</b></font>

```c
int pin = 13;           // LED on board
int interrupt0pin = 2;  // pin attached to int0 if you're using Uno
int buttonState;

void setup() {
  pinMode(pin, OUTPUT);
  pinMode(interrupt0pin, INPUT);

  // calls buttonUpdate() when button input changes (you can also use high-to-low or low-to-high)
  attachInterrupt(0, buttonUpdate, CHANGE);
}

void loop() {
  if (buttonState)
    digitalWrite(pin, HIGH);
  else
    digitalWrite(pin, LOW);
}

void buttonUpdate() {
  buttonState = digitalRead(interrupt0pin);
  delay(50);            // 50ms works for most switches - adjustable
}
```

<font size='4'><b>Lab</b></font>

This sketch monitors pulses on pin 2 and stores the duration in an array. When the array has been filled (64 pulses have been received), the duration of each pulse is displayed on the Serial Monitor:

```c
/*
Interrupts sketch
see Recipe 10.1 for connection diagram from Arduino Cookbook
*/

const int irReceiverPin = 2; //pin the receiver is connected to
const int numberOfEntries = 64;
volatile unsigned long microseconds;
volatile byte index = 0;
volatile unsigned long results[numberOfEntries];

void setup() {
  pinMode(irReceiverPin, INPUT);
  Serial.begin(9600);
  attachInterrupt(0, analyze, CHANGE); // encoder pin on interrupt 0 (pin 2);
  results[0] = 0;
}
void loop() {
  if (index >= numberOfEntries) {
    Serial.println("Durations in Microseconds are:") ;
    for ( byte i = 0; i < numberOfEntries; i++) {
      Serial.println(results[i]);
    }
    while (1);
  }
  delay(1000);
}
void analyze() {
  if (index < numberOfEntries ) {
    if (index > 0) {
      results[index] = micros() - microseconds;
    }
    index = index + 1;
  }
  microseconds = micros();
}
```

## 3.2. Timer interrupt

In [8]:
%%html
<iframe src="https://www.youtube.com/embed/m5_pFID-f-M?start=0&end=932" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

The Arduino UNO has three timers

- Timer0: An 8 bit timer used by Arduino functions `delay()`, `millis()` and `micros()`.
- Timer1: A 16 bit timer used by the `Servo()` library
- Timer2: An 8 bit timer used by the `Tone()` library

The Arduino has 3 Timers and 6 PWM output pins. The relation between timers and PWM outputs is:

- Pins 5 and 6: controlled by timer0
- Pins 9 and 10: controlled by timer1
- Pins 11 and 3: controlled by timer2

<font size='4'><b>Lab</b></font>

```c
#include "TimerOne.h"

void setup() {
  pinMode(13, OUTPUT);
  Timer1.initialize(500000);         // initialize timer1, and set a 500000 microseconds (or 1/2 second period)
  Timer1.attachInterrupt(timerIsr);  // attaches timerIsr() as a timer service routine
}

void timerIsr() {
  // Toggle LED
  digitalWrite(13, digitalRead(13) ^ 1);
}

void loop() {
  // your program here...
}
```

<font size='4'><b>Timer interrupt embeded functions</b></font>

- `millis()`
- `micro()`

<br>
<font size='4'><b>`millis()`</b></font>

Returns the number of milliseconds since the Arduino board began running the current program. This number will overflow (go back to zero), after approximately 50 days. 

```c
unsigned long time;

void setup(){
  Serial.begin(9600);
}
void loop(){
  Serial.print("Time: ");
  time = millis();
  //prints time since program started
  Serial.println(time);
  // wait a second so as not to send massive amounts of data
  delay(1000);
}
```

<font size='4'><b>`micros()`</b></font>

Returns the number of microseconds since the Arduino board began running the current program. This number will overflow (go back to zero), after approximately 70 minutes.


```c
unsigned long time;

void setup(){
  Serial.begin(9600);
}
void loop(){
  Serial.print("Time: ");
  time = micros();
  //prints time since program started
  Serial.println(time);
  // wait a second so as not to send massive amounts of data
  delay(1000);
}
```

__Note__

- 1,000 microseconds in a millisecond
- 1,000,000 microseconds in a second.

<font size='4'><b>Lab</b></font>

Blink without delay()

```c
/* Blink without Delay

 Turns on and off a light emitting diode (LED) connected to a digital
 pin, without using the delay() function.  This means that other code
 can run at the same time without being interrupted by the LED code.

 The circuit:
 * LED attached from pin 13 to ground.
 * Note: on most Arduinos, there is already an LED on the board
 that's attached to pin 13, so no hardware is needed for this example.

 created 2005
 by David A. Mellis
 modified 8 Feb 2010
 by Paul Stoffregen
 modified 11 Nov 2013
 by Scott Fitzgerald


 This example code is in the public domain.

 http://www.arduino.cc/en/Tutorial/BlinkWithoutDelay
 */

// constants won't change. Used here to set a pin number :
const int ledPin =  13;      // the number of the LED pin

// Variables will change :
int ledState = LOW;             // ledState used to set the LED

// Generally, you should use "unsigned long" for variables that hold time
// The value will quickly become too large for an int to store
unsigned long previousMillis = 0;        // will store last time LED was updated

// constants won't change :
const long interval = 1000;           // interval at which to blink (milliseconds)

void setup() {
  // set the digital pin as output:
  pinMode(ledPin, OUTPUT);
}

void loop() {
  // here is where you'd put code that needs to be running all the time.

  // check to see if it's time to blink the LED; that is, if the
  // difference between the current time and last time you blinked
  // the LED is bigger than the interval at which you want to
  // blink the LED.
  unsigned long currentMillis = millis();

  if (currentMillis - previousMillis >= interval) {
    // save the last time you blinked the LED
    previousMillis = currentMillis;

    // if the LED is off turn it on and vice-versa:
    if (ledState == LOW) {
      ledState = HIGH;
    } else {
      ledState = LOW;
    }

    // set the LED with the ledState of the variable:
    digitalWrite(ledPin, ledState);
  }
}
```

method 2: use timer interrupt

<font size='4'><b>Lab: LCD Timer</b></font>

```c
// include the library code:
#include <LiquidCrystal.h>

// initialize the library with the numbers of the interface pins
LiquidCrystal lcd(12, 11, 5, 4, 3, 2);

void setup() {
  // set up the LCD's number of columns and rows:
  lcd.begin(16, 2);
  // Print a message to the LCD.
  lcd.print("hello, world!");
}

void loop() {
  // set the cursor to column 0, line 1
  // (note: line 1 is the second row, since counting begins with 0):
  lcd.setCursor(0, 1);
  // print the number of seconds since reset:
  lcd.print(millis() / 1000);
}
```

<font size='4'><b>Lab: Dual 7 segments with timer interrupt</b></font>

In [9]:
%%html
<iframe src="https://www.youtube.com/embed/Y4qC8INSotM" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

# 4. Debouncing

## 4.1. Blinking LED using Button (Revisited)

<img src = "./image_files/d7.png" style="border:1px solid black", width =500>

```c
const int led    =   8;       //name pin 8 as led
const int button =   9;       //name pin 9 as button

void setup() {
  pinMode(led, OUTPUT);       //set pin 8 as OUTPUT
  pinMode(button, INPUT)  ;   //set pin 9 as INPUT
}

void loop() {
  int reads = digitalRead(button);  //read the digital value on pin 9
  digitalWrite(led, reads);         //set the digital output value of pin 8 to that value
}
```

- Pushbuttons often generate spurious open/close transitions when pressed, due to mechanical and physical issues: 

- these transitions may be read as multiple presses in a very short time fooling the program. 

- checking twice in a short period of time to make sure the pushbutton is definitely pressed.

<img src = "./image_files/debounceswitchb.jpg" style="border:1px solid black", width =500>


## 4.2. Interrupt-based debouncing

```c
int pin = 13;           // LED on board
int interrupt0pin = 2;  // pin attached to int0 if you're using Uno
int buttonState;

void setup() {
  pinMode(pin, OUTPUT);
  pinMode(interrupt0pin, INPUT);

  // calls buttonUpdate() when button input changes (you can also use high-to-low or low-to-high)
  attachInterrupt(0, buttonUpdate, CHANGE);
}

void loop() {
  if (buttonState)
    digitalWrite(pin, HIGH);
  else
    digitalWrite(pin, LOW);
}

void buttonUpdate() {
  buttonState = digitalRead(interrupt0pin);
  delay(50);            // 50ms works for most switches - adjustable
}
```

## 4.3. Software Debounce

https://www.arduino.cc/en/Tutorial/Debounce

```c
const int buttonPin = 2;    // the number of the pushbutton pin
const int ledPin = 13;      // the number of the LED pin

// Variables will change:
int ledState = HIGH;         // the current state of the output pin
int buttonState;             // the current reading from the input pin
int lastButtonState = LOW;   // the previous reading from the input pin
long lastDebounceTime = 0;  // the last time the output pin was toggled
long debounceDelay = 50;    // the debounce time; increase if the output flickers

void setup() {
  pinMode(buttonPin, INPUT);
  pinMode(ledPin, OUTPUT);

  // set initial LED state
  digitalWrite(ledPin, ledState);
}

void loop() {
  // read the state of the switch into a local variable:
  int reading = digitalRead(buttonPin);

  // check to see if you just pressed the button
  // (i.e. the input went from LOW to HIGH),  and you've waited
  // long enough since the last press to ignore any noise:

  // If the switch changed, due to noise or pressing:
  if (reading != lastButtonState) {
    // reset the debouncing timer
    lastDebounceTime = millis();
  }

  if ((millis() - lastDebounceTime) > debounceDelay) {
    // whatever the reading is at, it's been there for longer
    // than the debounce delay, so take it as the actual current state:

    // if the button state has changed:
    if (reading != buttonState) {
      buttonState = reading;

      // only toggle the LED if the new button state is HIGH
      if (buttonState == HIGH) {
        ledState = !ledState;
      }
    }
  }

  // set the LED:
  digitalWrite(ledPin, ledState);

  // save the reading.  Next time through the loop,
  // it'll be the lastButtonState:
  lastButtonState = reading;
}
```

In [10]:
%%html
<iframe src="https://www.youtube.com/embed/5XWgr4vJkQ4" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

## 4.4. Hardware debouncing (optional)

# 5. Synchronous Serial

In [11]:
%%html
<iframe src="https://player.vimeo.com/video/111224388" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

## 5.1. SPI

https://learn.sparkfun.com/tutorials/serial-peripheral-interface-spi?_ga=1.137392624.1133730960.1469244666

IMU sensors

In [12]:
%%html
<iframe src="https://player.vimeo.com/video/111225512" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

## 5.2 I2C

IMU sensors

In [13]:
%%html
<iframe src="https://player.vimeo.com/video/111239036" 
width="640" height="360" frameborder="0" allowfullscreen></iframe>

In [14]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>